In [1]:
import pandas as pd
import numpy as np
import verispy, os
from verispy import VERIS

[Source for VERISPY + Useful README](https://github.com/RiskLens/verispy)

In [3]:


DATA_DIR ="./data/validated/"
PKL_PATH = "./data/vdf.pkl"

v = VERIS(json_dir=DATA_DIR, verbose=True)

# json_to_df is an expensive function, so we save the converted dataframe for use between sessions. 
if not os.path.exists(PKL_PATH):
    print("we're making pickles")
    vdf = v.json_to_df()
    vdf.to_pickle(PKL_PATH)
vdf = pd.read_pickle(PKL_PATH)
# v.raw_df=vdf
print(vdf.shape)

v.enum_summary(vdf, "action")


Found 8323 json files.
(8323, 2347)


,enum,x,n,freq
0,Error,2418,8105.0,0.29833
1,Hacking,2213,8105.0,0.27304
2,Misuse,1697,8105.0,0.20938
3,Physical,1576,8105.0,0.19445
4,Malware,697,8105.0,0.08600
5,Social,568,8105.0,0.07008
6,Environmental,8,8105.0,0.00099
7,Unknown,218,NaN,NaN


##### v is a VERIS object. vdf contains the actual data. 

[Good StackOverflow Answer on working with Dataframes vvv](https://stackoverflow.com/questions/17071871/how-to-select-rows-from-a-dataframe-based-on-column-values)

In [4]:
df_impact = vdf.loc[:, vdf.columns.str.contains('rating')]
print(v.enum_summary(vdf, 'impact.overall_rating'))
print(v.enum_summary(vdf, "impact.loss.rating"))


# v.enum_summary(vdf, "impact.rating"

            enum     x      n     freq
0  Insignificant    52  116.0  0.44828
1    Distracting    25  116.0  0.21552
2       Damaging    20  116.0  0.17241
3        Painful    18  116.0  0.15517
4   Catastrophic     1  116.0  0.00862
5        Unknown  5767    NaN      NaN
       enum  x    n  freq
0     Major  0  0.0   NaN
1     Minor  0  0.0   NaN
2  Moderate  0  0.0   NaN
3      None  0  0.0   NaN
4   Unknown  0  NaN   NaN


In [5]:
df_action = vdf.loc[:, vdf.columns.str.startswith('action')]


In [6]:
df_final = df_action.iloc[:, :7]

In [7]:
df_bugs = vdf[vdf['action.Hacking'] | vdf['action.Malware']]
# print(df_bugs.shape)



In [9]:
indices = list(df_bugs.index)
bugs = [False] * df_final.shape[0]
for i in indices:
    bugs[i] = True

In [13]:
df_final['action.Bugs'] = bugs
df_final

,action.Environmental,action.Error,action.Hacking,action.Malware,action.Misuse,action.Physical,action.Social,action.Bugs
0,False,False,False,False,True,False,False,False
1,False,False,False,False,False,True,False,False
2,False,True,False,False,False,False,False,False
3,False,False,True,False,False,False,False,True
4,False,True,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...
8318,False,True,False,False,False,False,False,False
8319,False,False,False,False,True,False,False,False
8320,False,False,False,False,True,False,False,False
8321,False,False,False,False,False,True,False,False


In [11]:
#Creates a smaller table for ease of running
N = 1000
sample = np.random.choice(8323, N)
df_short = df_final.iloc[sample,:].reset_index().iloc[:,1:]


# Maybe we should id the column here in the future so we can refer to the original table values

In [21]:
impactful_df = vdf[vdf['impact.overall_rating.Insignificant'] | vdf['impact.overall_rating.Distracting'] | vdf['impact.overall_rating.Painful'] | vdf['impact.overall_rating.Catastrophic'] | vdf['impact.overall_rating.Damaging']].reindex()
impactful_df_bugs = impactful_df[impactful_df['action.Hacking'] | impactful_df['action.Malware']]
impactful_df_bugs


,action.Environmental,action.Error,action.Hacking,action.Malware,action.Misuse,action.Physical,action.Social,action.Unknown,action.environmental.notes,action.environmental.variety.Deterioration,...,victim.revenue.iso_currency_code.XPF,victim.revenue.iso_currency_code.YER,victim.revenue.iso_currency_code.ZAR,victim.revenue.iso_currency_code.ZMK,victim.revenue.iso_currency_code.ZWD,victim.secondary.amount,victim.secondary.notes,victim.secondary.victim_id,victim.state,victim.victim_id
38,False,False,True,False,False,False,False,False,NaN,False,...,False,False,False,False,False,NaN,NaN,NaN,CA,Uber
351,False,False,True,False,False,False,False,False,NaN,False,...,False,False,False,False,False,NaN,NaN,NaN,US-MI,Adrian College
370,False,False,True,False,False,False,True,False,NaN,False,...,False,False,False,False,False,NaN,NaN,NaN,US-IA,Virginia Gay Hospital
554,False,False,False,True,False,False,False,False,NaN,False,...,False,False,False,False,False,1.0,The dentist offices who were compromised could...,[Digital Dental Record],US-WI,PerCSoft
1166,False,False,True,True,False,False,True,False,NaN,False,...,False,False,False,False,False,NaN,NaN,NaN,CA-ON,Michael Garron Hospital
1215,False,False,True,False,True,True,False,False,NaN,False,...,False,False,False,False,False,NaN,NaN,NaN,WI,Brinks Security
1349,False,False,True,False,False,False,False,False,NaN,False,...,False,False,False,False,False,NaN,NaN,NaN,NaN,Society of Information Risk Analysts
1540,False,False,True,False,False,False,False,False,NaN,False,...,False,False,False,False,False,NaN,NaN,NaN,VA,"Albemarle County, VA"
1699,False,False,False,True,False,False,False,False,NaN,False,...,False,False,False,False,False,NaN,NaN,NaN,US-CT,Wolcott school district
1774,False,False,True,False,False,False,False,False,NaN,False,...,False,False,False,False,False,NaN,NaN,NaN,US-KY,Elmcroft Senior Living


In [22]:
indices = list(impactful_df_bugs.index)
print(indices)
bugs = [False] * impactful_df.shape[0]
for i in indices:
    print(i)
    bugs[i] = True

[38, 351, 370, 554, 1166, 1215, 1349, 1540, 1699, 1774, 2141, 2384, 2885, 2963, 3106, 3250, 3341, 4145, 4289, 4606, 4815, 4830, 4849, 5060, 5142, 5567, 5626, 5708, 6017, 6149, 6229, 6250, 6269, 6278, 6511, 6566, 6725, 6896, 6943, 7735, 7814, 7838, 8024, 8099, 8172, 8222, 8308]
38
351


IndexError: list assignment index out of range

In [12]:
df_short

,action.Environmental,action.Error,action.Hacking,action.Malware,action.Misuse,action.Physical,action.Social,action.Bugs
0,False,False,True,False,False,False,False,True
1,False,False,True,False,True,False,False,True
2,False,True,False,False,False,False,False,False
3,False,False,False,False,False,True,False,False
4,False,False,True,False,False,False,False,True
...,...,...,...,...,...,...,...,...
995,False,False,True,False,False,False,False,True
996,False,False,False,False,True,False,False,False
997,False,False,True,False,False,False,False,True
998,False,False,True,False,False,False,False,True
